# Problem 1
## a. Using variable elimination, calculate the probability that a student who did well on the exam understands the material.P(+u|+e). Show your work.

P(+e) = 

# Problem 2

In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
data = pd.read_csv("Life Expectancy Data.csv")
data.describe()

,Year,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
count,2938.000000,2928.000000,2928.000000,2938.000000,2744.000000,2938.000000,2385.000000,2938.000000,2904.000000,2938.000000,2919.000000,2712.00000,2919.000000,2938.000000,2490.000000,2.286000e+03,2904.000000,2904.000000,2771.000000,2775.000000
mean,2007.518720,69.224932,164.796448,30.303948,4.602861,738.251295,80.940461,2419.592240,38.321247,42.035739,82.550188,5.93819,82.324084,1.742103,7483.158469,1.275338e+07,4.839704,4.870317,0.627551,11.992793
std,4.613841,9.523867,124.292079,117.926501,4.052413,1987.914858,25.070016,11467.272489,20.044034,160.445548,23.428046,2.49832,23.716912,5.077785,14270.169342,6.101210e+07,4.420195,4.508882,0.210904,3.358920
min,2000.000000,36.300000,1.000000,0.000000,0.010000,0.000000,1.000000,0.000000,1.000000,0.000000,3.000000,0.37000,2.000000,0.100000,1.681350,3.400000e+01,0.100000,0.100000,0.000000,0.000000
25%,2004.000000,63.100000,74.000000,0.000000,0.877500,4.685343,77.000000,0.000000,19.300000,0.000000,78.000000,4.26000,78.000000,0.100000,463.935626,1.957932e+05,1.600000,1.500000,0.493000,10.100000
50%,2008.000000,72.100000,144.000000,3.000000,3.755000,64.912906,92.000000,17.000000,43.500000,4.000000,93.000000,5.75500,93.000000,0.100000,1766.947595,1.386542e+06,3.300000,3.300000,0.677000,12.300000
75%,2012.000000,75.700000,228.000000,22.000000,7.702500,441.534144,97.000000,360.250000,56.200000,28.000000,97.000000,7.49250,97.000000,0.800000,5910.806335,7.420359e+06,7.200000,7.200000,0.779000,14.300000
max,2015.000000,89.000000,723.000000,1800.000000,17.870000,19479.911610,99.000000,212183.000000,87.300000,2500.000000,99.000000,17.60000,99.000000,50.600000,119172.741800,1.293859e+09,27.700000,28.600000,0.948000,20.700000


# Divide the data into a training set and a testing set
I am dividing the training set and testing set by selecting 25% of the data for training (with replacement) and 75% as the test data set.


In [77]:
# To find correlation of each variable perform linear regression in relation to each variable
def print_col_correlations(data):
    filtered = data.values[:,:]
    filtered=filtered[:,3:]
    i = 3
    r_squared_data = []
    for col in filtered.T:
        col = col.astype(float)
        col = np.array(col)

        y =  data.values[:,3]

        y = y.astype(float)
        nans = np.isnan(y)
        y = y[~nans]

        col = col[~nans]

        col_nans = np.isnan(col)
        col=col[~col_nans]
        y = y[~col_nans]
        reg = LinearRegression().fit(col.reshape(-1,1), y)
        #print("score for: "+str(data.columns[i]))
        score = reg.score(col.reshape(-1,1), y)
        r_squared_data.append([data.columns[i],score])
        i+=1
    return pd.DataFrame(
        r_squared_data,columns=["measure_name","r squared"])
print_col_correlations(data)


,measure_name,r squared
0,Life expectancy,1.000000
1,Adult Mortality,0.484916
2,infant deaths,0.038635
3,Alcohol,0.163925
4,percentage expenditure,0.145820
5,Hepatitis B,0.065927
6,Measles,0.024833
7,BMI,0.322276
8,under-five deaths,0.049519
9,Polio,0.216742


Infant mortality ,Hepatitis B, Measles, under-five deaths, Total expenditure and Population	have low correlation with life expectancy. Therefore they will be removed from the analysis.

In [78]:
# To make model perform regresion on data set
